# Homework 3: N-Gram Language Models

## Total Points: 95 + 5 bonus
- **Overview**: In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this assignment, we will focus on the related problem of predicting the next character in a sequence given the previous characters.

- **Learning goals**:
    - Understand how to compute language model probabilities using maximum likelihood estimation.
    - Implement basic smoothing, back-off and interpolation.
    - Have fun using a language model to probabilistically generate texts.
    - Use a set of language models to perform text classification.

- **Data**: We will provide you with training and development data that has been manually labeled. We will also give you a test set without labels. You will build a classifier to predict the labels on our test set. You can upload your classifier’s predictions to Gradescope. We will score its predictions and maintain a leaderboard showing whose classifier has the best performance.

- **Delieverables:** This assignment has several deliverables:
    - Your implementations for the functions in the skeleton code (this notebook)
    - Answers to all questions labeled as `Answer #.#` in a file named `report.pdf`
    - Your model’s output for the test set (your model will be ranked on a leaderboard against the other students’ outputs)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings
- [Language Modeling with N-grams](https://web.stanford.edu/~jurafsky/slp3/3.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft) .
- [A Bit of Progress in Language Modeling](https://arxiv.org/abs/cs/0108005). Joshua Goodman. Computer Speech and Language .
- [The Unreasonable Effectiveness of Character-level Language Models](http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139). Yoav Goldberg. Response to Andrej Karpathy's blog post. 2015.
- [Language Independent Authorship Attribution using Character Level Language Models](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf). Fuchun Pen, Dale Schuurmans, Vlado Keselj, Shaojun Wan. EACL 2003.

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [1]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [2]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [3]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [4]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 63833115 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW3', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 63833115

Make sure this correct or we will not be able to store your grade


In [5]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Jonathon Michael Delemos'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [6]:
import math, random
from collections import Counter
from dill.source import getsource

In [7]:
## DO NOT CHANGE ANYTHING, JUST RUN

def get_class_source(cls):
    import re
    class_name = cls.__name__
    from IPython import get_ipython
    ipython = get_ipython()
    inputs = ipython.user_ns['In']
    pattern = re.compile(r'^\s*class\s+{}\b'.format(class_name))
    for cell in reversed(inputs):
        if pattern.search(cell):
            return cell
    return None

In [8]:
!gdown 19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T # https://drive.google.com/file/d/19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T/view?usp=share_link
!unzip -o cities_train.zip
!gdown 1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu # https://drive.google.com/file/d/1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu/view?usp=sharing
!unzip -o cities_val.zip
!gdown 1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1 # https://drive.google.com/file/d/1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1/view?usp=sharing
!gdown 11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4 # https://drive.google.com/file/d/11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4/view?usp=share_link
!gdown 19P37gR_XDup-h3a4LdKtQWbdM-lGPqin # https://drive.google.com/file/d/19P37gR_XDup-h3a4LdKtQWbdM-lGPqin/view?usp=sharing
!gdown 1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw # https://drive.google.com/file/d/1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T
To: /content/cities_train.zip
100% 160k/160k [00:00<00:00, 12.4MB/s]
Archive:  cities_train.zip
   creating: train/
  inflating: train/af.txt            
  inflating: train/de.txt            
  inflating: train/fi.txt            
  inflating: train/fr.txt            
  inflating: train/in.txt            
  inflating: train/ir.txt            
  inflating: train/cn.txt            
  inflating: train/za.txt            
  inflating: train/pk.txt            
Downloading...
From: https://drive.google.com/uc?id=1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu
To: /content/cities_val.zip
100% 7.56k/7.56k [00:00<00:00, 21.3MB/s]
Archive:  cities_val.zip
   creating: val/
  inflating: val/af.txt              
  inflating: val/cn.txt              
  inflating: val/de.txt              
  inflating: val/fi.txt              
  inflating: val/fr.txt              
  inflating: val/ir.txt              
  inflating: val/za.txt       

# Section 0: Generating N-Grams [5 points]
- **Problem 0:** Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the n characters preceding the current character. The sentence should be padded with n ~ characters at the beginning (we’ve provided you with `start_pad(n)` for this purpose). If n=0 , all contexts should be empty strings. You may assume that n≥0.

```
>>> ngrams(1, 'abc')
[('~', 'a'), ('a', 'b'), ('b', 'c')]

>>> ngrams(2, 'abc')
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
```

In [9]:
### DO NOT CHANGE ###
def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n
### DO NOT CHANGE ###

In [10]:
#returns ngram, next char

def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''


# This makes a lot more sense now

#  "~~~" → "d"

# "~~d" → "o"

# "~do" → "g"

    ## AUTOGRADER HELPER ##
    def start_pad(n):
        ''' Returns a padding string of length n to append to the front of text
            as a pre-processing step to building n-grams '''
        return '~' * n
    ## AUTOGRADER HELPER ##
    my_list = []
    word = start_pad(n) + text
    counter = 0
    saved = text
    # Areas for improvement: didn't know it would extend to cdef - figure out why
    for i in range(len(text)):
      # 0:4 = ~~~~ , 1:5 ~~~a great solution
      startpad= word[i:i+n]
      # final char position
      final_char = word[i+n]
      my_list.append((startpad, final_char))
    return my_list


In [11]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q0_ngrams', answer = getsource(ngrams))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 1: Creating an N-Gram Model [36 points]

1.   List item
2.   List item


In this section, you will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, `NumPy` and `NLTK`.


In [12]:

class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k):
        # About `k`: you don't need to worry about it for problem 1.1, it is something you will need in problem 2.1
        self.k = k

        self.contexts = {}

        ## TODO 1.1 - see problem 1.1
        self.n = n
        self.vocab = set()


    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        return self.vocab

    def get_context(self):
        ''' Returns the dictionary for context '''
        return self.contexts

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        ## TODO 1.2 - see problem 1.2This is the function,
        # ngrams returns a ngram and char
        n_grams = ngrams(self.n,text)
        # here, we are checking the n_grams from our n_gram generator
        for key, pair in n_grams:
          # if the n_gram key doesn't exist, let's first add it to the nested dictionary
          if key not in self.contexts:
            # initialize the key with an empty nested dictionary structure
            self.contexts[key] = {'count':0,'chars':{}}


          # append the char
          # add the char to the nested key,pair

          self.contexts[key]['chars'][pair] = self.contexts[key]['chars'].get(pair,0) + 1
          # increment the count
          self.contexts[key]['count'] +=1
          self.vocab.add(pair)

    def prob(self, context, char):
          ''' Returns the probability of char appearing after context '''

          #this is a little extreme to add smoothing before the next section where it's covered?
          if context not in self.contexts:
            if len(self.vocab) == 0:
              return 1
            else:
              return 1/len(self.vocab)
          denom = self.contexts[context]['count']
          num = self.contexts[context]['chars'].get(char,0)
          return (num + self.k)/(denom + self.k * len(self.vocab))


    def random_char(self, context):
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        ## TODO 1.3 - see problem 1.3
        # random number between 0 and 1
        random_probability = random.random()
        # prob_list = []
        # list_char = self.contexts[context]['chars'].keys()
        # for x in list_char:
        #   prob_list.append(self.prob(context, x))


        # never seen before case
        sum_prob = 0.0
        list_prob = []
        vocab = sorted(self.vocab)
        for char in vocab:
          sum_prob += self.prob(context, char)
          if random_probability < sum_prob:
            return char
        return vocab[-1]

    # creates a random char text append function by generating a random value and picking a letter in a set based off that value
    def random_text(self, length):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        ## TODO 1.4 - see problem 1.4

        # use the same sliding glass door approach as above
        char_pred = []
        word = start_pad(self.n)
        for i in (range(length)):
          if self.n == 0:
            random_char = self.random_char('')
            char_pred.append(random_char)
          else:
            startpad= word[-self.n:]
            random_char = self.random_char(startpad)
            word = word+random_char
            char_pred.append(random_char)
        return ''.join(char_pred)

    # normalized inverse probability - finds each characters probability and adds them together in log space
    def perplexity(self, text):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        # reaching into the dict to grab the probabilities
        N=0
        log_prob = 0
        # reaching into the dict to grab the probabilities
        for context, char in ngrams(self.n, text):
          # finding the probability of that char
          char_prob = self.prob(context, char)
          # if char prob == 0, we avoid that case and report inf
          if char_prob == 0:
            return float('inf')
          # running prob log times
          log_prob = log_prob + math.log(char_prob)
          N+=1

        return math.exp(-log_prob/N)



## 1.1 Initialization [3 points]
In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order n of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the `set` of all characters used by this model).

- **Problem 1.1:** finish `__init__(self, n, k)` and `get_vocab(self)` [3 points]

In [13]:
# PennGrader - DO NOT CHANGE
ngram_model = NgramModel(1, 0)
print(ngram_model.get_vocab())
grader.grade(test_case_id = 'test_q11_init_get_vocab', answer = get_class_source((NgramModel)))

set()
Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Update and Calculate Probabilities [15 points]
Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be 1/V where V is the size of the vocab.

- **Problem 1.2:** finish `update(self, text)` and `prob(self, context, char)` [15 points]

In [14]:
# PennGrader - DO NOT CHANGE
all_test_cases = [
    (1, ['abab', 'abcd'], [('b','c'), ('a', '~')]),
    (2, ['ababa', 'abcdddd', 'babcdabag'], [('ab', 'a'), ('ad', 'b')]),
    (3, ['ababa', 'abcdddd', 'babcdabag'], [('ddd', 'd'), ('~~c', 'f')]),
    (2, ['abc'], [('~a', 'b'), ('~~', 'a')])
]

all_test_results = []
for test_data in all_test_cases:
    nm = NgramModel(test_data[0], 0)
    for text in test_data[1]:
        nm.update(text)
    vocab = nm.get_vocab()
    probs = []
    for prob_check in test_data[2]:
        prob = nm.prob(prob_check[0], prob_check[1])
        probs.append(prob)
    all_test_results.append((vocab, probs))

# reload_grader()
grader.grade(test_case_id = 'test_q12_update_prob', answer = all_test_results)

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.3 Random Character [6 points]

Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $r$ be a random number between 0 and 1. Your method should return the character $v_i$ such that:
$$
\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).
$$
You should use a single call to the `random.random()` function to generate $r$, for example:
```
>>> m = NgramModel(0, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> [m.random_char('') for i in range(25)]
['a', 'c', 'c', 'a', 'b', 'b', 'b', 'c', 'a', 'a', 'c', 'b', 'c', 'a', 'b', 'b', 'a', 'd', 'd', 'a', 'a', 'b', 'd', 'b', 'a']
```

- **Problem 1.3:** finish `random_char(self, context)` [6 points]

In [15]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [2, 3, 4]
random_char_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_char = [nm.random_char('') for i in range(25)]
    random_char_lst.append(random_char)

grader.grade(test_case_id = 'test_q13_rand_char', answer = random_char_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.4 Random Text [6 points]
In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> m.random_text(25)
abcdbabcdabababcdddabcdba
```

- **Problem 1.4:** finish `random_text(self, length)` [6 points]

In [16]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [42, 43, 44]
random_text_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_text = nm.random_text(25)
    random_text_lst.append(random_text)

grader.grade(test_case_id = 'test_q14_rand_text', answer = random_text_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.5 Writing Shakespeare [6 points]

Now you can train a language model. We have provided this corpus of Shakespeare at `shakespeare_input.txt` (if you cannot find it, re-run the `Setup 2: Dataset / Packages` section).

We’ve also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided and `create_ngram_model_lines(model_class, path, n, k)` that will create and return an n-gram model trained line-by-line on the file path provided. You should use the first for the Shakespeare file and the second for the city name files.

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

```python
>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
>>> m.random_text(250)
```

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? Is it saying *First, I will generate the word "First"*?  No, probably not.  Explain what is going on in your writeup.

- **Answer 1.5:** Generate Shakespear texts with at least 3 numbers of n and discuss on the results [6 points]
  - Write your response to this in your pdf

In [17]:
## DO NOT CHANGE ##
def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

In [18]:
# Example
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
m.random_text(250)

'First Citizen:\nThis is the sad and we been impostor that sets up\nhis head, and Warwick knew it all of good?\n\nOCTAVIUS CAESAR:\nI must not his\nname out such a lose and sudden command,\nIf crooked sadly themselves alone?\n\nLUCIUS:\nMad for the powers? show'

In [19]:
## Answer 1.5 in your PDF

# Section 2: Smoothing, Perplexity, and Interpolation [30 points]

In this part of the assignment, you'll adapt your code in order to implement several of the  techniques described in [Section 3.5 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

## 2.1 Smoothing [6 points]

Laplace Smoothing is described in section 3.5.1 of the book. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section *Add-k 3.5.2*.

```python
>>> m = NgramModel(1, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('a', 'a')
0.14285714285714285
>>> m.prob('a', 'b')
0.5714285714285714
>>> m.prob('c', 'd')
0.4
>>> m.prob('d', 'a')
0.25
```

- **Problem 2.1**: Update your `NgramModel` code from Part 1 to implement add-k smoothing. (see comment of `## TODO 2.1`) [6 points]




In [20]:
# PennGrader - DO NOT CHANGE
smoothing_test_cases = [
    (1, 1, ['abab', 'abcd'], ('b', 'a')),
    (2, 1, ['ababa', 'abcdddd', 'babcdabag'], ('ba', 'b')),
    (3, 2, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], ('aba', 'b'))
]
smoothing_test_results = []
for i, j, corpus, test_words in smoothing_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    smoothing_test_results.append(nm.prob(test_words[0], test_words[1]))

grader.grade(test_case_id = 'test_q21_smoothing', answer = smoothing_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 2.2 Perplexity [12 points]

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as **extrinsic** evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. **Intrinsic** evaluation: Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $W = w_1 w_2 ... w_N$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.perplexity('abcd')
1.189207115002721
>>> m.perplexity('abca')
inf
>>> m.perplexity('abcda')
1.515716566510398
```

- **Problem 2.2**: implement the `perplexity(self, text)` function in `NgramModel` [6 points]

In [21]:
# PennGrader - DO NOT CHANGE
ppl_test_cases = [
    (1, 0, ['abab', 'abcd'], 'abca'),
    (2, 0, ['ababa', 'abcdddd', 'babcdabag'], 'ba'),
    (3, 1, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], 'abcdddddddddd')
]
ppl_test_results = []
for i, j, corpus, test_words in ppl_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    ppl_test_results.append(nm.perplexity(test_words))

grader.grade(test_case_id = 'test_q22_ppl', answer = ppl_test_results)




Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.2:** Compare and discuss the perplexity for text that is similar and different from Shakespeare's plays. We provide you two dev text files, a New York Times article (`nytimes_article.txt`) and several of Shakespeare's sonnets (`shakespeare_sonnets.txt`); if you cannot find them, re-run `Setup 2: Dataset / Package` at the beginning. Also, feel free to experiment with your own text. [6 points]

In [22]:



## Answer 2.2 in your PDF
for i in range(2,5,1):
    print(i)
    m = NgramModel(i, 1)
    with open('nytimes_article.txt', 'r', encoding='latin-1') as f:
        text = f.read()
        #print(text)
    with open('shakespeare_input.txt', 'r', encoding='latin-1') as g:
        shake = g.read()
    m.update(shake)
    print(m.perplexity(text))

for i in range(2,5,1):
    print(i)
    m = NgramModel(i, 1)
    with open('shakespeare_input.txt', 'r', encoding='latin-1') as g:
        shake = g.read()
    with open('shakespeare_sonnets.txt', 'r', encoding='latin-1') as f:
        text = f.read()
        #print(text)
    m.update(shake)
    print(len(m.get_vocab()))
    print(m.perplexity(text))



2
11.366820578456359
3
10.075639427063924
4
11.125617645800839
2
67
7.910100720888994
3
67
6.122231996035264
4
67
6.485626437020775


## 2.3 Interpolation [12 points]

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

```python
>>> m = NgramModelWithInterpolation(1, 0)
>>> m.update('abab')
>>> m.prob('a', 'a')
0.25
>>> m.prob('a', 'b')
0.75

>>> m = NgramModelWithInterpolation(2, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('~a', 'b')
0.4682539682539682
>>> m.prob('ba', 'b')
0.4349206349206349
>>> m.prob('~c', 'd')
0.27222222222222225
>>> m.prob('bc', 'd')
0.3222222222222222
```
- **Problem 2.3**: implement the `NgramModelWithInterpolation` class [6 points]

In [23]:
class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, n, k):
        super().__init__(n,k)
        self.models = [NgramModel(i,k) for i in range(n+1)]
        self.lambdas = [1/(n+1)] * (n+1)


    def set_lambdas(self, new_lambdas):
        if len(new_lambdas) != len(self.models):
            raise ValueError("Must provide exactly {} lambdas".format(len(self.models)))
        total = sum(new_lambdas)
        if total == 0:
            raise ValueError("Lambdas must not all be zero")
        self.lambdas = [x / total for x in new_lambdas]

    def update(self, text):
        for m in self.models:
            m.update(text)
            self.vocab |= m.get_vocab()


    def prob(self, context, char):
        if len(self.vocab) == 0:
            return 1.0
        total = 0.0
        for j, m in enumerate(self.models, start=1):
            sub_context = context[-(j-1):] if j > 1 else ""
            total += self.lambdas[j-1] * m.prob(sub_context, char)
        return total


In [24]:
# PennGrader - DO NOT CHANGE
interpolation_test_cases = [
    [(0, 1), ['abab'], ('a', 'a')],
    [(2, 1), ['abab', 'abcd'], ('~c', 'd')],
    [(2, 1), ['abeadab', 'abcd', 'eadb'], ('~a','b')],
    [(2, 1), ['abab', 'abcd', 'aaaaad'], ('bc', 'd')],
]

interpolation_test_results = []
for (n, k), corpus, (context, char) in interpolation_test_cases:
    nm_interpolation = NgramModelWithInterpolation(n, k)
    for text in corpus:
        nm_interpolation.update(text)
    interpolation_test_results.append(nm_interpolation.prob(context, char))

grader.grade(test_case_id = 'test_q23_interpolation', answer = interpolation_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.3**: Experiment with a few different lambdas and values of k and discuss the effects of smoothing and interpolation in terms of perplexity. Alternatively, you can also try different number of n. You may still use text files from 2.2. [6 points]



In [25]:
## Answer 2.3 in your PDF


m = NgramModelWithInterpolation(3,3)
with open('shakespeare_input.txt', 'r', encoding='latin-1') as g:
    shake = g.read()
with open('nytimes_article.txt', 'r', encoding='latin-1') as f:
    text = f.read()
    #print(text)
m.update(shake)
print(m.perplexity(text))


n = NgramModel(3, 3)
with open('shakespeare_input.txt', 'r', encoding='latin-1') as f:
    shake = f.read()
with open('nytimes_article.txt', 'r', encoding='latin-1') as g:
    text = g.read()
    #print(text)
n.update(shake)
print(len(n.get_vocab()))
print(n.perplexity(text))

11.583458092719079
67
10.733513927179741


# Section 3: Text Classification using N-Grams [20 points + 5 bonus]
**No PennGrader in this section, see `Deliverables` for grading details**
## Overview
Language models can be applied to text classification. If we want to classify a text $D$ into a category $c \in C={c_1, ..., c_N}$. We can pick the category $c$ that has the largest posterior probability given the text. That is,

$$ c^* = arg max_{c \in C} P(c|D) $$

Using Bayes rule, this can be rewritten as:

$$ c^* = arg max_{c \in C} P(D|c) P(c)$$

If we assume that all classes are equally likely, then we can just drop the $P(c)$ term:

$$ = arg max_{c \in C} P(D|c)$$

Here $P(D \mid c)$ is the likelihood of $D$ under category $c$, which can be computed by training language models for all texts associated with category $c$. This technique of text classification is drawn from [literature on authorship identification](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf), where the approach is to learn a separate language model for each author, by training on a data set from that author. Then, to categorize a new text D, they use each language model to calculate the likelihood of D under that model, and pick the  category that assigns the highest probability to D.


## Try it!
We have provided you training and validation datsets consisting of the names of cities. The task is to predict the country a city is in. The following countries are including in the dataset.

```
af	Afghanistan
cn	China
de	Germany
fi	Finland
fr	France
in	India
ir	Iran
pk	Pakistan
za	South Africa
```

We'll set up a leaderboard for the text classification task. **Your job is to configure a set of language models that perform the best on the text classification task.** We will use the `city names` dataset, which you should have already downloaded. The test set has one unlabeled city name per line.

Your code should output a file `test_labels.txt` with one two-letter country code per line.

Feel free to extend the `NgramModel` or `NgramModelWithInterpolation` when creating your language model. Possible ideas to consider and experiment with when creating your model are utilizing a special end-of-text character, trying a new method for determining the vocab, and improving how your model handles novel characters.

## Deliverables
- **Answer 3.1:** Train your best model on the given training data (located in `train` folder), and report accuracy on the given validation data (located in `val` folder) for your final model. In order to receive full credit, your model must be able to outperform all the baselines. [5 points]

- **Answer 3.2:** Describe the parameters you used for the final submission, such as n , k and interpolation. Discuss how you made the decision and why your combination have better results, we encourage using a table to display all validation accuracies of your experiment process. Be sure to include a detailed error analysis: identify several examples of cities on which your final model is making errors, discuss potential reasons. [15 points]

- **Answer 3.3:** Make predictions on the given test data (`cities_test.txt`) and upload `test_labels.txt` to Gradescope, which contains one two-letter country code per line. Please note that testing accuracy should be valid and outperform the baseline in order to receive full credits (included in 3.1). Top 3 will be granted 5 bonus points.

In [26]:
COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

In [47]:

from google.colab import drive
drive.mount('/content/drive')

import os
!ls /content/drive
!ls /content/drive/MyDrive

# grabbing the cities to work with
train_dir = "/content/drive/MyDrive/labels"
val_dir= "/content/val/"


## parsing dataset cities
cities_test = []
with open("/content/drive/MyDrive/cities_test.txt", "r", encoding='latin-1') as f:
    for line in f:
        cities_test.append(line.strip())


## parsing data set training
train_test = dict()
for text in os.listdir("/content/train/"):
    base_city = []
    with open(f"/content/train/{text}", "r", encoding='latin-1') as f:
        for line in f:
            base_city.append(line.strip())
    train_test[os.path.splitext(text)[0]] = base_city

## parsing data set validation
val_set = dict()
for text in os.listdir('/content/val/'):
    base_city = []
    with open(f"/content/val/{text}", "r", encoding='latin-1') as f:
        for line in f:
            base_city.append(line.strip())
        val_set[os.path.splitext(text)[0]] = base_city

#now we have a list of countries as keys and cities as their data in the dict


# let's build the models using this technique. We are going to create a single nm model, then loop through and train the model on the cities insiess their training folder
def build_model_list(n, k, text, train_test):
    model_list = {}
    # we are building one model per country trained on it'own data. It's a bit confusing
    # because we don't have access to the label markers i.e. 1,0, suppose country name is the 1,0, but it correlates to a much larger set of data. so a single one for 500 data points
    # annoying
    for country in COUNTRY_CODES:
        nm = NgramModelWithInterpolation(n, k)
        if country in train_test:
          for city in train_test[country]:
              nm.update(city)
        model_list[country] = nm
    find_perplexity(model_list, val_set)



# slightly different approach toward perplexity and ngrams? Idk. Buidling is like searching for something in a fog.
def find_perplexity(model_list, val_set):
    predict = dict()

    total = 0
    correct = 0
    city = ''
    for country_name, country_data in val_set.items():
        for city in country_data:
          list_perplexities= {}
          for model in model_list:
            if len(model_list[model].vocab) == 0:
              score = float("inf")   # or a large number so it won’t get picked
            else:
              list_perplexities[model] = model_list[model].perplexity(city)
              if model_list[model].perplexity(city) > 50:
                print((model, city))
          # print(max(list_perplexities, key = list_perplexities.get))
          predicted = min(list_perplexities, key=list_perplexities.get)
          if predicted == country_name:
                correct += 1
          if predicted != country_name:
                print(list_perplexities['de'])
          total += 1
    accuracy = correct / total
    print("Validation accuracy:", accuracy)

    ideas = {}
    for country in COUNTRY_CODES:
      n = NgramModelWithInterpolation(3,.868)
      if country in train_test:
        for city in train_test[country]:
          n.update(city)
      ideas[country] = n

    predict_test_labels(model_list, cities_test, "/content/test_labels.txt")


def predict_test_labels(model_list, test_cities, output_path="test_labels.txt"):
    with open(output_path, "w", encoding="utf-8") as f:
        for city in test_cities:
            list_perplexities = {}
            for model in model_list:
                if len(model_list[model].vocab) == 0:
                    score = float("inf")
                else:
                    score = model_list[model].perplexity(city)
                list_perplexities[model] = score

            # pick model (country) with lowest perplexity
            predicted_country = min(list_perplexities, key=list_perplexities.get)

            # write its two-letter code
            f.write(predicted_country + "\n")


build_model_list(3, .6, cities_test, train_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive  Shareddrives
 cities_test.txt				 labels
'Colab Notebooks'				 report.gdoc
'Copy of CIS530 HW2: Submission Template.gdoc'	 test_labels.txt
 greatgatsby.txt				 writeup.gdoc
14.401572167921374
9.914979100787493
18.265252536519437
13.691938939700453
16.273028922658504
13.327339653005184
19.61720298401024
13.748560916210002
25.815752268725927
15.209854409105
18.341426489671345
26.142566359065555
15.531573250479486
18.97858014173504
21.087870685660196
19.596409960892046
12.73910151449576
14.15688880039775
22.37813093771817
12.882175611106119
16.54852412848008
22.205315689489467
16.946483178083195
26.142652787888778
20.172375183288764
9.975115339872922
18.46135363556762
14.154647772645909
14.397997025138539
16.01910707449429
13.373589649692136
23.555076597354116
22.242055973203023
17.315252801477527
18.362375079783885
15.656584447208258
22.062368488411124

In [28]:
## Answer 3.1 and 3.2 in your PDF ###

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework3.ipynb` and `homework3.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`
- Your `report.pdf`
  - including answers to 1.5, 2.2, 2.3, 3.1, and 3.2
- `test_labels.txt` from `Section 3`